In [1]:
from utils import utils,learner
from utils import PlantPathologyDataset as dataset
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

/home/braedon/.local/lib/python3.8/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 7.1.0. Several security issues (CVE-2020-11538, CVE-2020-10379, CVE-2020-10994, CVE-2020-10177) have been fixed in pillow 7.1.0 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [2]:
dtype = torch.FloatTensor
fixed_model_base = nn.Sequential(
    nn.Conv2d(3, 5, kernel_size=7, stride=1),
    nn.ReLU(inplace=True),
    nn.BatchNorm2d(5),
    nn.MaxPool2d(2, stride=2),
    learner.Flatten(),
    nn.Linear(128050, 1024),
    nn.ReLU(inplace=True),
    nn.Linear(1024, 12),
)

fixed_model = fixed_model_base.type(dtype)

loss_fn = nn.CrossEntropyLoss().type(dtype)
optimizer = optim.Adadelta(fixed_model_base.parameters(), lr = 0.05)
#

In [3]:
thomas = False
if thomas:
    csv_file='C:\\Users\\tjtom\PycharmProjects\PlantPathology2021\\train.csv'
    root_dir='C:\\Users\\tjtom\PycharmProjects\PlantPathology2021\\train_images'
    mappings_dir='C:\\Users\\tjtom\PycharmProjects\PlantPathology2021\\labelMappings.csv'
else:
    csv_file='./train.csv'
    root_dir='./train_images'
    mappings_dir='./labelMappings.csv'

plant_dataset = dataset.PlantPathologyDataset(csv_file=csv_file,
                                              root_dir=root_dir,
                                              mappings_dir=mappings_dir)
train_data = DataLoader(plant_dataset, batch_size=10, sampler= learner.ChunkSampler(75, 0))
validation_data = DataLoader(plant_dataset, batch_size=10, sampler=learner.ChunkSampler(25, 75))
learner.train(fixed_model_base,train_data,loss_fn,optimizer,print_every=5)
learner.check_accuracy(fixed_model_base, validation_data)

Starting epoch 1 / 1
torch.Size([10, 3, 400, 267]) tensor([ 3, 11,  9,  9,  0,  3,  6,  3,  0,  3])
torch.Size([10, 3, 400, 267]) tensor([0, 9, 3, 9, 1, 9, 9, 9, 3, 9])
torch.Size([10, 3, 400, 267]) tensor([4, 6, 9, 9, 3, 9, 6, 3, 0, 9])
torch.Size([10, 3, 400, 267]) tensor([3, 1, 9, 6, 9, 3, 3, 0, 0, 4])
torch.Size([10, 3, 400, 267]) tensor([ 3,  9, 11,  1,  4,  9,  3,  1,  1,  4])
t = 5, loss = 13.5665
torch.Size([10, 3, 400, 267]) tensor([10,  9,  3,  9,  6,  1,  1,  0,  3,  1])
torch.Size([10, 3, 400, 267]) tensor([9, 9, 2, 3, 1, 9, 1, 6, 4, 9])
torch.Size([5, 3, 400, 267]) tensor([9, 6, 9, 0, 9])
Checking accuracy on validation set
Got 6 / 25 correct (24.00)


0.24